In [1]:
import tensorflow as tf
import keras
import numpy as np
from keras.preprocessing.text import Tokenizer
import pronouncing
import gensim.downloader
import pickle
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

2022-12-10 13:16:38.886073: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


KeyboardInterrupt: 

In [ ]:
## CONFIGURE STARTER WORD AND RHYME SCHEME HERE

next_char = ['‘twas']
rhyme_scheme = 'ABBCBB'

In [6]:
with open('tokenizer', 'rb') as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)

model = keras.models.load_model('model_poems')

IndexError: Read less bytes than requested

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model

  def generate_one_step(self, inputs, states=None):
    
    temp = tokenizer.texts_to_sequences(inputs)
    if len(temp[0]) == 0:
      temp = [[1]]
    input_ids = np.array(temp)
    predicted_logits, states = self.model(input_ids, states=states,
                                          return_state=True)
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature

    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    predicted_chars = tokenizer.sequences_to_texts([predicted_ids.numpy()])
    
    return predicted_chars, states

In [ ]:
def get_rhyme_with_max_embedding(base_word, context_words):
  
  all_rhyme_words = pronouncing.rhymes(base_word)

  best_count = float('-inf')
  best_word = ""
  for r in all_rhyme_words:
    count = 0
    seen_content = 1
    for c in context_words:
      if r in glove_vectors and c[0] in glove_vectors:
        count += glove_vectors.similarity(r, c[0])
        seen_content += 1
    
    count = count / seen_content
    
    
    if count >= best_count:
      best_word = r
      best_count = count

  return best_word

In [ ]:
one_step_model = OneStep(model)
states = None

result = [next_char]

breaks = rhyme_scheme.count(" ")
lines = len(rhyme_scheme) - breaks
n = 0
prev_char = []


while n < lines:
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  
  if next_char == ['\n']:
    if prev_char != ['\n']:
      result.append(next_char)
      n += 1
  else:
    result.append(next_char)
    
  prev_char = next_char

In [ ]:
line_count = 0
rhymes = {}

temp_scheme = rhyme_scheme.replace(" ", "")
i = 0
start_index = 0
for i in range(len(result)):
  if result[i] == ['\n']:
    rhyme_index = temp_scheme[line_count]
    if rhyme_index in rhymes:
      base_rhyme_word = rhymes[rhyme_index]
      embedding_words = result[start_index:i-1]
      rhymed_word = get_rhyme_with_max_embedding(base_rhyme_word[0], embedding_words)
      result[i-1] = [rhymed_word]
      rhymes[rhyme_index] = result[i-1]
      
    else:
      rhymes[rhyme_index] = result[i-1]
    
    start_index = i + 1
    line_count += 1
    
    
start_index = 0
line_count = 0
for i in range(len(result)):
  if result[i] == ['\n']:
    if rhyme_scheme[line_count] == " ":
        result[start_index - 1] = ['\n\n']
        
    line_count += 1
    start_index = i + 1

In [ ]:
poem = " "
for l in result:
  poem += l[0]
  poem += " "
print(poem)